# LSTM

RNNの学習においては勾配消失問題を解決するためには、RNNレイヤのアーキテクチャを根本から変える必要があります。

ここで登場するのは、シンプルのRNNを改良した「ゲート付きRNN」です。この「ゲート付きRNN」では多くのアーキテクチャが提案されており、その代表的な仕組みはLSTM（Long Short-Term Memory）になります。

## LSTMのインタフェース

### LSTMの全体像

RNNとLSTMレイヤのインタフェースの違いは、LSTMには$c$という経路があることです。この$c$は記憶セルと呼ばれ、これを通じてネットワークを流れる情報の流れを制御します。セル状態は、ネットワークの一種の「記憶」であり、重要な情報を長期間にわたって保持する能力を持っています。これにより、LSTMは長期間にわたる依存関係を捉えることができます。

$c_t$には、時刻$t$におけるLSTMの記憶が格納されています。具体的に言えば、現在の記憶セル$c_t$は、三つの入力$(c_{t-1},h_{t-1},x_t)$から何らかの計算によって求められています。そのため、これに過去から時刻$t$までにおいて必要な情報が全て格納されていると考えられます。

必要な情報が詰まった記憶を元に、外部のレイヤへ隠れ状態$h_t$を出力します。


![](./Figure/lstm1.png)

### ゲート機構

ゲートは、セル状態に流れる情報を制御するために使用される仕組みです。

ゲートの「開き具合」は$0.0 - 1.0$までの実数で表されます。そしてその数値によって、必要な情報を保持し、不要な情報を排除し、適切な時に適切な情報を出力することができます。

ここで重要なのは、ゲートの「開き具合」ということは、データから自動的に学ばせるということです。

## LSTMの構造

```{margin}
$\odot$はアダマール積（要素ごとの積）を表します。シグモイド関数によって得られたゲートの値$0_t$と、tanh関数によって得られたセル状態$c_t$の正規化された値を要素ごとに掛け合わせることを意味します。
```

![](./Figure/lstm2.png)

```{note}

- tanh関数:tanh関数は出力を$-1$から$1$の範囲に正規化します。この数値は、何らかのエンコードされた「情報」に対する度合いが表されていると解釈できます。

- シグモイド関数: シグモイド関数は出力を0から1の範囲に正規化します。データをどれだけ通すかという割合を表しますので、各ゲート（忘却ゲート、入力ゲート、出力ゲート）で情報の流れを制御するのに適しています。

そのため、ゲートではシグモイド関数、実質的な「情報」を持つデータにはtanh関数が活性化関数として使われます。

```

### forgetゲート

LSTMのforgetゲートでは、長期記憶から不要な情報を忘却するための制御を行っている。

ここで、forgetゲートで行う一連の計算を$\sigma$で表すことにします。計算は次の式で表されます。$\sigma$はsigmoid関数を表します。

$$f_t = \sigma(x_tW_x^{(f)}+h_{t-1}W_h^{(f)} + b^{(f)})$$


- 入力
    - 現在の入力$x_t$
    - 前の時点の隠れ層の出力$h_{t-1}$

- 重みとバイアス
    - 現在の入力$x_t$に適用されるゲートの重み行列$W_x^{(f)}$
    - 前の時点の隠れ層の出力$h_{t-1}$に適用されるゲートの重み行列$W_h^{(f)}$

み付けされた入力とバイアスの合計にシグモイド関数によって計算されるため、$0$ から $1$ の値をとります。

![](./Figure/forget.png)

### inputゲート

新しい情報を追加する際、何も考えずに追加するのではなく、追加する情報としてどれだけ価値があるかを判断する上で、追加する情報を選択します。これにより、長期間にわたる依存関係をより効果的に管理し、複雑なシーケンスデータを扱うことができるようになります。

具体的には、inputゲートによって重みつけされた情報が新たに追加されることになります。

$$i_t = \sigma(x_tW_x^{(i)}+h_{t-1}W_h^{(i)} + b^{(i)})$$


![](./Figure/input_gate.png)

### 新しい記憶セル

LSTMではセルの長期記憶を保つための変数$c_t$が用意されています。長期記憶$c_t$に対して、古くなった記憶を削除したり、新しい情報を新規追加したりすることで、適当な長期記憶を可能にしています。

具体的には、

```{margin}
新しい「情報」を記憶セルに追加することを目的としていますので、「ケード」ではなく$tanh$ノードが使われます。
```

- 入力ゲートの計算
    - 現在の入力$x_t$と前の時点の隠れ状態$h_{t-1}$から、inputゲート$i_t$がシグモイド関数を用いて計算されます。
    - 同時に、tanh関数を用いて新しい候補セル状態$\tilde{c}_t$が生成されます。

$$
g = tanh(x_tW_x^{(g)}+h_{t-1}W_h^{(g)} + b^{(g)})
$$

- セル状態の更新
    - inputゲート$i_t$と新しい候補セル状態$\tilde{c}_t$がアダマール積によって組み合わされます。
    - forgetゲート$f_t$を用いて、前のセル状態$c_{t-1}$が更新されます。
- 最終的なセル状態
    - 更新された前のセル状態と新しく生成されたセル状態が加算され、新しいセル状態$c_t$が生成されます。

![](./Figure/long_memory.png)

### outputゲート

outputゲートは隠れ状態$h_t$の形成に使用されます。

現在の記憶セル$c_t$は、$(c_{t-1},h_{t-1},x_t)$を入力として求められます。そして、更新された$c_t$を使って、隠れ状態の$h_t$計算されます。ここで、$tanh(c_t)$の各要素に対して、「それらが次時刻の隠れ状態としてどれだけ重要か」ということを調整します。

なお、outputゲートの開き具合は、入力$x_t$と前の状態$h_{t-1}$から求めます。

$$o_t = \sigma(x_tW_x^{(o)}+h_{t-1}W_h^{(o)} + b^{(o)})$$

$sigmoid$関数による出力と$tanh$関数によるセル状態の出力を掛け合わせ、新しい隠れ状態$h_t$を生成します。

$$
h_t = o_t \odot \tanh(c_t)
$$




![](./Figure/output_gate.png)

```{note}
LSTMは、勾配消失問題を軽減するように設計されています。

- ゲート機構
    - LSTMのゲートは、セル状態に流れる情報を精密に制御します。セル状態のゲート制御された更新は、勾配が長いシーケンスを通じても安定して伝播するのを支援し、勾配消失問題の軽減に貢献します。また、各時点での情報の重要性を評価し、必要な情報を保持しながら不要な情報を取り除くことで、ニューラルネットワークがより効率的に学習し、安定した勾配を維持できるようにします。
    - forgetゲートが「忘れるべき」と判断した記憶セルの要素に対しては、その勾配の要素は小さくなります。その一方で、forgetゲートが「忘れではいけない」と導いた要素に対しては、その勾配の要素は劣化することなく過去方向へ伝わります。そのため、記憶セルの勾配は、勾配消失は起こさずに伝播することが期待できるのです。

- 非線形性の制御
    - RNNの逆伝播で、「行列の積」を繰り返し行っていましたので、勾配消失が起きたのです。
    - LSTMでは、「行列の積」の計算ではなく「要素ごとの積」で逆伝播行います。特に、シグモイドやtanhのような非線形活性化関数は、ゲートの開閉を制御しますが、セル状態自体の更新は加算によって行われるため、勾配が長いシーケンスを通しても安定して伝播し続けることができます。

```